# Progetto 2021

In [1]:
SEED = 9246


import os
import json
import time
from datetime import datetime
import collections
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import models
import warnings
import os, shutil
import pickle
from imutils import paths
import cv2
import sys
from skimage import io, color
import pickle

from PIL import Image

from skimage import io
import pandas as pd
import numpy as np
np.random.seed(SEED)

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.metrics import classification_report

import torchsummary

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


CUDA = torch.cuda.is_available()
device = torch.device("cuda" if CUDA else "cpu")

if CUDA:
    print('run on cuda')


warnings.filterwarnings("ignore")

In [2]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


# Dataset

In [3]:
DATASET_DIR = '/content/drive/MyDrive/ProgettoAnalisiImmagini/progetto_2021_dataset' #INSERIRE PATH DATASET GENERALE

#UTILIZZARE dizionarioFeatureTest per diminuire la durata del test
#dizionarioFeatureTest contiene per ogni trailer le feature associate gia calcolate in precedenza
DRIVE_FOLDER = '/content/drive/MyDrive/ProgettoAnalisiImmagini/progetto_2021_saves/model_checkpoint' #INSERIRE PATH IN CUI SI TROVA IL FILE dizionarioFeatureTest e model.pth



JSON_DATA = os.path.join(DATASET_DIR, 'train_test_split_dict.json')

with open(JSON_DATA) as fp:
    dataset_json = json.load(fp)


In [4]:
if not os.path.exists(DRIVE_FOLDER):
  os.mkdir("/content/drive/MyDrive/ProgettoAnalisiImmagini/progetto_2021_saves")
  os.mkdir("/content/drive/MyDrive/ProgettoAnalisiImmagini/progetto_2021_saves/model_checkpoint")



CHECKPOINT = os.path.join(DRIVE_FOLDER, 'model.checkpoint')
MODELFILE = os.path.join(DRIVE_FOLDER, 'model.pth')



CHECKPOINTLoss = os.path.join(DRIVE_FOLDER, 'traningLosses')

CHECKPOINTDizTrain = os.path.join(DRIVE_FOLDER, 'dizionarioFeatureTrain')
CHECKPOINTDizTest = os.path.join(DRIVE_FOLDER, 'dizionarioFeatureTest')



def save_dizFeature(dizFeature,train):
  if train:
    torch.save(dizFeature, CHECKPOINTDizTrain)
    print("!!DIZIONARIO TRAIN FEATURE SALVATO!!")
  else:
    torch.save(dizFeature, CHECKPOINTDizTest)
    print("!!DIZIONARIO TEST FEATURE SALVATO!!")


    
def load_dizFeature(train):
  if train:
    if not os.path.exists(CHECKPOINTDizTrain):
        return None
    return torch.load(CHECKPOINTDizTrain)  
  else:
    if not os.path.exists(CHECKPOINTDizTest):
        return None
    return torch.load(CHECKPOINTDizTest)  




def save_traningLoss(traningLossModel):
    torch.save({
            'traningLossesModel': traningLossModel
            }, CHECKPOINTLoss)
    print(" !!Traning Loss Salvata!!")
    
  


def load_traningLoss():
    if not os.path.exists(CHECKPOINTLoss):
        return None
    traninglossModel = torch.load(CHECKPOINTLoss)['traningLossesModel']
    return traninglossModel

    

def save_checkpoint(epoch, model, optimizer, loss):
    torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            }, CHECKPOINT)
    

def load_checkpoint(model, optimizer):
    if not os.path.exists(CHECKPOINT):
        return None, None
    checkpoint = torch.load(CHECKPOINT)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']

    return epoch, loss

def save_model(model):
    torch.save(model.state_dict(), MODELFILE)
    print("Modello Salvato")


def load_model(model):
    if os.path.exists(MODELFILE):
        model.load_state_dict(torch.load(MODELFILE))



def load_ImgMigliori():
         if os.path.exists("/content/drive/MyDrive/progetto_2021_saves/cacheDatasetTrain"):
          return "/content/drive/MyDrive/progetto_2021_saves/cacheDatasetTrain"
          

       return None

In [5]:
labels = set()
#Numero tag 85


for k in dataset_json.values():
    for lable_list in k.values():
        for v in lable_list:
            labels.add(v)

            


            
label_idx = {v: i for i, v in enumerate(sorted(labels))}







## Caricamento Dati

In [6]:
class VideoDataset2(Dataset):


    def __init__(self, dataset_folder, labels_dict,dizFeature):
        """
        Args:
            dataset_folder (string): Path to the folder with mp4 files.
            labels_dict (dict): dict filename - list of label.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.labels_dict = labels_dict
        self.root_dir = dataset_folder
        self._files = np.array(list(self.labels_dict.keys()))
        self.dizFeature=dizFeature


    def __len__(self):
        return len(self.labels_dict)

    def __getitem__(self, idx):
          name = self._files[idx]
  
          

          lab = torch.zeros(len(label_idx), dtype=torch.float32)
          for label in self.labels_dict[name]:
            lab[label_idx[label]] = 1

          feat=self.dizFeature[name]/10
          
          return feat,lab



datasetTestOttimizzato = VideoDataset2(DATASET_DIR, dataset_json['test'] ,load_dizFeature(False))


In [7]:
batch_size = 29



testDataLoaderOttimizzato = torch.utils.data.DataLoader(datasetTestOttimizzato, 
                                                batch_size=batch_size, 
                                                shuffle=False
                                                 )

In [ ]:
testSize=len(datasetTestOttimizzato)
print("TEST SIZE: ",testSize,"NUMERO DI BATCH:",len(testDataLoaderOttimizzato))

TEST SIZE:  1112 NUMERO DI BATCH: 39


# Loading model

Il nostro modello è composto da:  


*   Il livello (feature) di una Vgg16 pre-Addestrata che ci permette di calcolare le featureMap per ogni trailer.

*  Un livello fully-connected composto da dei livelli lineare per effettuare la classificazione.





In [8]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def test(net,img_size=52):
    x = torch.randn(1,3,img_size,img_size)
    y = net(x)

    print("Input: {}; Output: {}; N. of params: {:,}".format(list(x.size()), list(y.size()), count_parameters(net)))

In [9]:
from torchvision import models
from torch import optim, cuda

vgg = models.vgg16(pretrained=True, progress=False)

resNet = models.resnet50(pretrained=True, progress=False)


for param in vgg.parameters():
    param.requires_grad = False


for param in resNet.parameters():
    param.requires_grad = False


vgg.classifier=nn.Sequential()
resNet.fc=nn.Sequential()


vgg = vgg.to(device)
resNet=resNet.to(device)






Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


In [ ]:
print("VGG")
test(vgg,52)
print("ResNet50")
test(resNet,64)

VGG
Input: [1, 3, 52, 52]; Output: [1, 25088]; N. of params: 0
ResNet50
Input: [1, 3, 64, 64]; Output: [1, 2048]; N. of params: 0


In [10]:
import torch.nn.functional as F


class AzGmNet(nn.Module):
    def __init__(self):
        super(AzGmNet, self).__init__()
        
        self.featureVgg=vgg.features
         
        self.featureResNet=resNet.forward
        
        
        #self.lstm=nn.LSTM(input_size=2560, hidden_size=128, num_layers=3,batch_first=True)


        self.classifier = nn.Sequential(
            nn.Linear(6656 , 1024),
            nn.ReLU(True),
            nn.Linear(1024 , 1024),
            nn.ReLU(True),
            nn.Linear(1024 , 85),
          
            
        )

        

    def forward(self, x):
        #x(Batch size, numImgTrailer,Canali,H,W). y(batchSize,85).

        #(batch*numImgTrailer,canali,H,W)
        xNew=x.view(x.shape[0]*x.shape[1],x.shape[2],x.shape[3],x.shape[4])
        xNew=xNew.to(device)


        outputFeatureResNet=self.featureResNet(xNew)
        outputFeatureVgg=self.featureVgg(xNew)
   
      

        
        #(batch,numImgTrailer,Feature)      
        outputFeatureVgg=outputFeatureVgg.view(x.shape[0],x.shape[1],-1)
        outputFeatureResNet=outputFeatureResNet.view(x.shape[0],x.shape[1],-1)

    

        #(batch,FeatureTotali)
        fmTotVgg=torch.zeros((outputFeatureVgg.shape[0],outputFeatureVgg.shape[2])).to(device)
        fmTotResNet=torch.zeros((outputFeatureResNet.shape[0],outputFeatureResNet.shape[2])).to(device)
        

        for i in range(0,outputFeatureVgg.shape[0]):
          fmTotVgg[i,:]=outputFeatureVgg[i,:,:].sum(dim=0) #MEDIA DELLE FEATURE DI OGNI TRAILER
          fmTotResNet[i,:]=outputFeatureResNet[i,:,:].sum(dim=0) #MEDIA DELLE FEATURE DI OGNI TRAILER
             
        fmTot = torch.cat((fmTotVgg,fmTotResNet),1)

        output= self.classifier(fmTot)

        return output,fmTot
  
    

In [11]:
model=AzGmNet()
model=model.to(device)
optimizer = optim.Adam(model.parameters())
load_model(model)

# Test

In [12]:
#TEST OTTIMIZZATO

def value(prediction,y):
  y_true, predictions = torch.cat(y, axis=0), torch.cat(prediction, axis=0)
  report = classification_report(y_true, predictions,target_names=list(sorted(label_idx.keys())))

  return report
  
def test():
  topk=10
  predictions = []
  y_true = []

  model.eval()
  batch=1


  with torch.no_grad():


      for feature,la in testDataLoaderOttimizzato: 



          feature = feature.to(device)
          la=la.to(device)


          logps=model.classifier(feature)
          
          
          y_pred = torch.sigmoid(logps)
      

          _, idx = y_pred.topk(topk, dim=1)

          y_pred = torch.zeros_like(y_pred)
          y_pred.scatter_(1, idx, 1)
          predictions.append(y_pred.cpu())

          y_true.append(la.cpu())

    
  return value(predictions,y_true)



In [13]:
print(test())
print()
print("Soglie")
print("BassLine 1  8pts  micro-precision>=0.10   micro-recall>=0.45  micro-f1=0.20  ")  
print("BassLine 2  7pts  micro-precision>=0.15   micro-recall>=0.50  micro-f1=0.25")  
print()

                      precision    recall  f1-score   support

                LGBT       0.08      0.43      0.14        61
              action       0.19      0.96      0.32       191
     action_comedies       0.00      0.00      0.00         6
           adventure       0.09      0.87      0.16        75
   alcohol_addiction       0.00      0.00      0.00         6
               alien       0.00      0.00      0.00        12
           animation       0.21      0.86      0.34        21
            aviation       0.06      0.40      0.11        20
        bank_robbery       0.25      0.05      0.09        19
              biopic       0.07      0.49      0.13        59
             cartoon       0.12      0.25      0.16         8
               chase       0.00      0.00      0.00        16
            children       0.00      0.00      0.00        12
              comedy       0.19      0.97      0.31       211
        comedy_drama       0.13      0.31      0.18        65
       